<a href="https://colab.research.google.com/github/EyalKoubi/CloudProject/blob/main/CloudGamePrototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
cd /content/drive/MyDrive

In [1]:
!pip install firebase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
################ imports #########################################################################
import ipywidgets as widgets
from IPython.display import display,HTML
import random
from firebase import firebase
import time
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.widgets import Button
import tkinter as tk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from IPython.display import clear_output
from IPython.display import display
import io
import IPython.display
from matplotlib.figure import Figure

# Initialize the Firebase connection
FBconn = firebase.FirebaseApplication('https://new-proj-917e8-default-rtdb.firebaseio.com/', None)

############################### get function #######################################################

# Function to retrieve all questions from Firebase
def get_all_questions():
    result = FBconn.get('/Questions', None)
    questions = []
    if result:
        for key, value in result.items():
            options = [value['Option1'],value['Option2'],value['Option3'],value['Option4']]
            question = {
                'question': value['Question'],
                'options': options,
                'correctOption': value['Correct']
            }
            questions.append(question)
    return questions

####################################### statistic screen ######################################################

def statisticScreen(right_answers, response_times, total_time):
    # Calculate the average response time for right answers
    right_response_times = [t for t in response_times if t >= 0]
    avg_response_time = np.mean(right_response_times)

    # Generate the data for the bar chart
    categories = list(range(1, len(response_times) + 1))
    response_times = [t if t >= 0 else np.nan for t in response_times]

    # Create the figure and axes for the graph
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(categories, response_times, color='lightblue', alpha=0.8)

    # Set axes labels and title
    ax.set_xlabel('Question')
    ax.set_ylabel('Response Time (seconds)')
    ax.set_title('Quiz Game Results')

    # Customize the graph appearance
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)

    # Save the current figure to a BytesIO object
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    # Create an IPython Image from this object
    plot_img = IPython.display.Image(buf.read(), format='png')
    buf.close()

    # Create a widgets.Image, ready to be displayed
    plot_widget = widgets.Image(value=plot_img.data, format='png')

    # Create the text annotations for details
    points_text = f'Points: {right_answers}'
    total_time_text = f'Total Time: {total_time} seconds'
    if right_response_times != []:
        fastest_time_text = f'Fastest Time: {min(right_response_times)} seconds'
    else:
        fastest_time_text = f'Fastest time: None'
    avg_time_text = f'Average Time: {avg_response_time:.2f} seconds'
    details_widget = widgets.HTML(f"<div style='display: inline-block; border-style: dotted; border-color: RoyalBlue; background-color: lightblue; color: black; margin: 15px; padding:25px; font-size:24px;'> {points_text}&nbsp;&nbsp;&nbsp;{total_time_text}<br>{fastest_time_text}&nbsp;&nbsp;&nbsp;{avg_time_text} </div>")
    details_widget.add_class("details")

    # Display the image and details in a vertical box
    graph_details_container = widgets.VBox([plot_widget, details_widget], layout=widgets.Layout(height='auto', justify_content='center', align_items='center'))

    # Close the figure
    plt.close(fig)

    # Create the Back and Try Again buttons
    btn_back = widgets.Button(description='Back', layout=widgets.Layout(width='250px', height='auto'))
    btn_back.add_class("button")
    btn_try_again = widgets.Button(description='Try Again', layout=widgets.Layout(width='250px', height='auto'))
    btn_try_again.add_class("button")

    # Function to handle button clicks
    def handle_button_click(b):
        if b.description == 'Back':
            clear_output(wait=True)
            display(createPlayerScreen())
        elif b.description == 'Try Again':
            clear_output(wait=True)
            quizScreen()

    # Assign the button click handlers
    btn_back.on_click(handle_button_click)
    btn_try_again.on_click(handle_button_click)

    # Create a container for the buttons
    buttons_container = widgets.HBox([btn_back, btn_try_again])

    # main_container.children = [container,buttons_container]
    main_container = widgets.VBox([graph_details_container, buttons_container], layout=widgets.Layout(width='100%', height='auto', justify_content='center', align_items='center'))
    main_container.add_class("background-image")

    # Apply CSS styles
    css = """
    <style>
      .question {
        color: #4169E1;
        text-align: center;
        font-size: 30px;
        padding: 50px 50px;
        text-transform: uppercase;
    }

    .details {
        align: center;
    }

    .button {
      display: inline-block;
      padding: 15px 25px;
      font-size: 24px;
      cursor: pointer;
      text-align: center;
      text-decoration: none;
      outline: none;
      color: #F0F8FF;
      background-color: #6495ED;
      border: none;
      border-radius: 15px;
      box-shadow: 0 9px #4682B4;
      margin: 25px 50px 75px 100px
    }

    .button:hover {background-color: #F0F8FF; color: #6495ED;  margin: 25px 50px 75px 100px}

    .button:active {
      background-color: #4169E1;
      color: #F0F8FF;
      box-shadow: 0 5px #B0E0E6;
      margin: 25px 50px 75px 100px;
      transform: translateY(4px);
    }

    .background-image {
        background-image: url('https://w.forfun.com/fetch/3d/3dd0b8f4ab9d0752d3ca9a569723efb3.jpeg?w=1470&r=0.5625');
        background-repeat: no-repeat;
        background-position: center;
        background-size: cover;
    }
    </style>
    """

    # Return the main_container
    return main_container



####################################### quiz screen ######################################################

def quizScreen():
    # Create a submit button
    submit_button = widgets.Button(description='Submit', layout=widgets.Layout(width='auto', height='auto'))
    submit_button.add_class("button")

    # Create a question widget
    question_widget = widgets.HTML()
    question_widget.add_class("question")

    # Create a radio button widget
    options_widget = widgets.RadioButtons(layout={'width': 'max-container'})
    options_widget.layout.justify_content = 'center'
    options_widget.add_class("options")

    options_widget.style.description_width = 'initial'
    options_widget.style.font_size = '50px'

    # Create a box to hold the question and options
    question_container = widgets.VBox([question_widget, options_widget], layout=widgets.Layout(justify_content='center', align_items='center'))

    # Create a list to store burndown icons
    burndown_icons = []

    # Create a burndown graph widget
    burndown_graph = widgets.HTML()
    burndown_graph.layout.margin = '10px'

    # Get all questions
    questions = get_all_questions()

    # Shuffle the questions randomly
    random.shuffle(questions)

    # Set the initial question index to 0
    current_question_index = 0

    # Create an array to store the time taken for each question
    time_array = [-1] * len(questions)

    # Create a variable to store the total time taken
    total_time = 0

    # Function to update the question and options
    def update_question():
        nonlocal current_question_index
        nonlocal total_time

        # Check if all questions have been answered
        if current_question_index >= len(questions):
            time.sleep(1)
            num_correct = 0  # Variable to store the number of correct answers

            for i in range(len(burndown_icons)):
                total_time += time_array[i]
                if burndown_icons[i] == 'X':
                    time_array[i] = -1
                else:
                    num_correct += 1
            total_time = round(total_time, 2)
            screen_container.close()
            display(statisticScreen(num_correct, time_array, total_time))
            return

        # Get the current question and options
        question = questions[current_question_index]['question']
        options = questions[current_question_index]['options']

        # Update the question widget and options widget
        question_widget.value = f"<p>{question}<p>"
        options_widget.options = options

        # Reset the options widget selection
        options_widget.value = None

        # Function to calculate and update the time taken for the current question
        def update_time(change):
            nonlocal current_question_index

            if current_question_index < len(questions) and options_widget.value is not None:
                current_time = time.time() - start_time
                time_array[current_question_index] = round(current_time, 2)

        # Register the update_time function to be called when the options are changed
        options_widget.observe(update_time, 'value')

        # Start the timer for the current question
        start_time = time.time()

    # Function to handle button click event
    def submit_button_clicked(button):
        nonlocal current_question_index

        # Check if an option has been selected
        if options_widget.value is None:
            return

        # Check if the selected option is correct
        selected_option = options_widget.value
        correct_option = questions[current_question_index]['correctOption']
        is_correct = selected_option == correct_option

        # Add 'V' or 'X' to burndown_icons based on correctness
        burndown_icons.append('V' if is_correct else 'X')

        # Increment the current_question_index
        current_question_index += 1

        # Update the burndown graph
        update_burndown_graph()

        # Update the question and options
        update_question()

        # Function to update the burndown graph
    def update_burndown_graph():
        graph_html = ''
        for icon in burndown_icons:
            if icon=='V':
              graph_html += '<span style="margin-right:7px;"><img src="https://github.com/EyalKoubi/CloudProject/blob/main/rightAnswer.png?raw=true" align_items="center", width=50px, height=50px></span>'
            elif icon == 'X':
              graph_html += '<span style="margin-right:7px;"><img src="https://github.com/EyalKoubi/CloudProject/blob/main/wrongAnswer.png?raw=true" align_items="center", width=50px, height=50px></span>'
        graph_html += ''
        burndown_graph.value = graph_html

    # Update the question and options initially
    update_question()

    # Create a box to hold the burndown graph and submit button
    bottom_container = widgets.HBox([submit_button])
    bottom_container.layout.justify_content = 'center'

    # Create a box to hold the entire screen
    screen_container = widgets.VBox([burndown_graph, question_container, bottom_container],layout=widgets.Layout(width='100%', height='900px', justify_content='center', align_items='center'))
    screen_container.add_class("background-image")

    # Display the screen container
    display(screen_container)

    # Register the button click event handler
    submit_button.on_click(submit_button_clicked)

    # Apply CSS styles
    css = """
    <style>
      .question {
        color: #4169E1;
        text-align: center;
        font-size: 20px;
        padding: 50px 50px;
        text-transform: uppercase;
      }

      .button {
        display: inline-block;
        padding: 15px 25px;
        font-size: 24px;
        cursor: pointer;
        text-align: center;
        text-decoration: none;
        outline: none;
        color: #F0F8FF;
        background-color: #6495ED;
        border: none;
        border-radius: 15px;
        box-shadow: 0 9px #4682B4;
        margin: 25px 50px 75px 100px
      }

      .button:hover {background-color: #F0F8FF; color: #6495ED;  margin: 25px 50px 75px 100px}

      .button:active {
        background-color: #4169E1;
        color: #F0F8FF;
        box-shadow: 0 5px #B0E0E6;
        margin: 25px 50px 75px 100px;
        transform: translateY(4px);
      }
    
      .background-image {
          background-image: url('https://w.forfun.com/fetch/3d/3dd0b8f4ab9d0752d3ca9a569723efb3.jpeg?w=1470&r=0.5625');
          background-repeat: no-repeat;
          background-position: center;
          background-size: cover;
      }
    </style>
    """
    display(HTML(css))


####################################### welcome screen #################################################### 

def createWelcomeScreen():
    # Create main logo widget
    logo_widget = widgets.HTML("<img src='https://github.com/EyalKoubi/CloudProject/blob/main/headindaclouds.png?raw=true' align_items='center', width='184.5px', height='108.5px'>")
    logo_widget.add_class("main-logo")

    # Create main title widget
    title_widget = widgets.HTML("<img src='https://github.com/EyalKoubi/CloudProject/blob/main/welcomMsg.png?raw=true' align_items='center'>")
    title_widget.add_class("main-title")

    # # Create second title widget
    # subtitle_widget = widgets.HTML("<div style='text-align: center;'><h2 style='color: Violet; font-family: monaco, monospace;'>It's like Squid Game, but here<br>you stay alive :)</h2></div>")
    # subtitle_widget.add_class("subtitle")

    def handle_manager_button_click(b):
      screen_container.close()
      display(createPopup(screen_container))

    def handle_player_button_click(b):
      screen_container.close()
      display(createPlayerScreen())

    # Create manager button widget
    manager_button = widgets.Button(description="Manager", layout=widgets.Layout(width='250px', height='auto'))
    manager_button.add_class("button")
    manager_button.on_click(handle_manager_button_click)

    # Create player button widget
    player_button = widgets.Button(description="Player", layout=widgets.Layout(width='250px', height='auto'))
    player_button.add_class("button")
    player_button.on_click(handle_player_button_click)

    # Create button container widget
    button_container = widgets.HBox([player_button, manager_button])
    button_container.add_class("button-container")

    # Create screen container widget
    screen_container = widgets.VBox([logo_widget, title_widget, button_container], layout=widgets.Layout(width='100%', height='900px', justify_content='center', align_items='center'))
    screen_container.add_class("background-image")

    # Apply CSS styles for  background image and rounded buttons
    css = """
    <style>
    .main-logo {
      position: absolute;
      top: 20px;
    }

    .main-title {
      margin-bottom: 100px;
      margin-top: 100px
    }
    
    .button {
      display: inline-block;
      padding: 15px 25px;
      font-size: 24px;
      cursor: pointer;
      text-align: center;
      text-decoration: none;
      outline: none;
      color: #F0F8FF;
      background-color: #6495ED;
      border: none;
      border-radius: 15px;
      box-shadow: 0 9px #4682B4;
      margin: 25px 50px 75px 100px
    }

    .button:hover {background-color: #F0F8FF; color: #6495ED;  margin: 25px 50px 75px 100px}

    .button:active {
      background-color: #4169E1;
      color: #F0F8FF;
      box-shadow: 0 5px #B0E0E6;
      margin: 25px 50px 75px 100px;
      transform: translateY(4px);
    }
    
    .background-image {
        background-image: url('https://w.forfun.com/fetch/3d/3dd0b8f4ab9d0752d3ca9a569723efb3.jpeg?w=1470&r=0.5625');
        background-repeat: no-repeat;
        background-position: center;
        background-size: cover;
    }
    </style>
    """
    display(HTML(css))

    return screen_container

####################################### popup screen ###############################################################

def createPopup(screen_container):
    overlay = widgets.Output()

    def handle_exit_button_click(b):
        overlay.close()
        display(createWelcomeScreen())

    def handle_submit_button_click(b):
        choice = password_input.value
        if choice != "shablool":
            error_label.value = "<p style='color:red; font-size:18px'>The choice is incorrect, please try again</p>"
        else:
            screen_container.close()
            overlay.close()
            display(createManagerScreen())

    with overlay:
        exit_button = widgets.Button(description="X", layout=widgets.Layout(width='auto', height='auto', align_self='flex-end'))
        exit_button.add_class("button")
        exit_button.on_click(handle_exit_button_click)

        password_input = widgets.Password(placeholder="Enter password")
        password_container = widgets.HBox([password_input], layout=widgets.Layout(width='100%', justify_content='center', align_items='center'))

        error_label = widgets.HTML(value="<p style='color:red;'></p>")

        submit_button = widgets.Button(description="Submit", layout=widgets.Layout(width='250px', height='auto', justify_content='center', align_items='center'))
        submit_button.add_class("button")
        submit_button.on_click(handle_submit_button_click)

        content = widgets.VBox([exit_button, password_container, error_label, submit_button], layout=widgets.Layout(width='100%', height='auto', justify_content='center', align_items='center'))
        content.add_class("background-image")

        css = """
        <style>
        .button {
            display: inline-block;
            padding: 15px 25px;
            font-size: 24px;
            cursor: pointer;
            text-align: center;
            text-decoration: none;
            outline: none;
            color: #F0F8FF;
            background-color: #6495ED;
            border: none;
            border-radius: 15px;
            box-shadow: 0 9px #4682B4;
        }

        .button:hover {
            background-color: #F0F8FF;
            color: #6495ED;
        }

        .button:active {
            background-color: #4169E1;
            color: #F0F8FF;
            box-shadow: 0 5px #B0E0E6;
            transform: translateY(4px);
        }

        .background-image {
            background-image: url('https://w.forfun.com/fetch/3d/3dd0b8f4ab9d0752d3ca9a569723efb3.jpeg?w=1470&r=0.5625');
            background-repeat: no-repeat;
            background-position: center;
            background-size: cover;
            padding-top: 10px;
            padding-right: 10px;
        }
        </style>
        """

        display(widgets.VBox([widgets.HTML(css), content]))

    return overlay

####################################### add quiz screen #######################################################################

def createAddQuiz():
    global overlay  # Declare overlay as a global variable

    def handle_exit_button_click_to_manager(b):
        form.close()
        overlay.close()
        display(createManagerScreen())

    form = widgets.VBox(layout=widgets.Layout(width='100%', height='auto', justify_content='flex-start', align_items='center'))
    form.add_class("background-image")

    exit_button = widgets.Button(description="X", layout=widgets.Layout(width='auto', height='auto', align_self='flex-end', flex='1 1 auto'))
    exit_button.add_class("button")
    exit_button.on_click(handle_exit_button_click_to_manager)

    question_input = widgets.Text(placeholder="Question")
    option1_input = widgets.Text(placeholder="Option 1")
    option2_input = widgets.Text(placeholder="Option 2")
    option3_input = widgets.Text(placeholder="Option 3")
    option4_input = widgets.Text(placeholder="Option 4")
    correct_input = widgets.Dropdown(options=["Option 1", "Option 2", "Option 3", "Option 4"])
    submit_button = widgets.Button(description="Submit", layout=widgets.Layout(width='250px', height='auto'))
    submit_button.add_class("button")
    error_label = widgets.HTML(value="One of the fields is empty.", layout={'display': 'none', 'color': 'red'})
    success_label = widgets.HTML(value="The quiz added successfully!", layout={'display': 'none', 'color': 'green'})

    # Create the add question container
    add_question = widgets.HTML("<img src='https://github.com/EyalKoubi/CloudProject/blob/main/addQuestion.png?raw=true' align_items='center'>")
    add_question_container = widgets.HBox([add_question], layout=widgets.Layout(justify_content='center', align_items='center', margin='20px'))

    form.children += (exit_button, add_question_container, question_input, option1_input, option2_input, option3_input, option4_input, correct_input,
                      submit_button, error_label, success_label)

    def handle_submit_button_click(b):
        question = question_input.value
        option1 = option1_input.value
        option2 = option2_input.value
        option3 = option3_input.value
        option4 = option4_input.value
        options = [option1, option2, option3, option4]
        correct = int(correct_input.value.split(" ")[1])

        if not question or not option1 or not option2 or not option3 or not option4 or not correct:
            error_label.layout.display = 'block'
            success_label.layout.display = 'none'
        else:
            # Check if the question already exists
            existing_questions = FBconn.get('/Questions', None)  # Assuming 'questions' is the path to the question data in your Firebase database

            if existing_questions:
                existing_questions = existing_questions.values()
                if any(q['Question'] == question for q in existing_questions):
                    error_label.value = "The question already exists."
                    error_label.layout.display = 'block'
                    success_label.layout.display = 'none'
                    return

            # Add the quiz to the database
            data = {'Question': question, 'Option1': options[0], 'Option2': options[1], 'Option3': options[2], 'Option4': options[3], 'Correct': options[correct-1]}
            result = FBconn.post('/Questions', data)  # Assuming 'questions' is the path to the question data in your Firebase database

            if result:
                error_label.layout.display = 'none'
                success_label.layout.display = 'block'
                question_input.value = ""
                option1_input.value = ""
                option2_input.value = ""
                option3_input.value = ""
                option4_input.value = ""
            else:
                error_label.value = "Failed to add the quiz to the database. Please try again."

        # Apply CSS styles
        css = """
        <style>

        .title {
          color: #4169E1;
          text-align: center;
          font-size: 30px;
          padding: 50px 50px;
          text-transform: uppercase;
        }

        .button {
          display: inline-block;
          padding: 15px 25px;
          font-size: 24px;
          cursor: pointer;
          text-align: center;
          text-decoration: none;
          outline: none;
          color: #F0F8FF;
          background-color: #6495ED;
          border: none;
          border-radius: 15px;
          box-shadow: 0 9px #4682B4;
          margin: 25px 50px 75px 100px
        }

        .button:hover {background-color: #F0F8FF; color: #6495ED; margin: 25px 50px 75px 100px}

        .button:active {
          background-color: #4169E1;
          color: #F0F8FF;
          box-shadow: 0 5px #B0E0E6;
          margin: 25px 50px 75px 100px;
          transform: translateY(4px);
        }

        .background-image {
            background-image: url('https://w.forfun.com/fetch/3d/3dd0b8f4ab9d0752d3ca9a569723efb3.jpeg?w=1470&r=0.5625');
            background-repeat: no-repeat;
            background-position: center;
            background-size: cover;
        }
        </style>
        """

    submit_button.on_click(handle_submit_button_click)

    overlay.close()
    display(form)

####################################### remove quiz screen #######################################################################

def createRemoveQuiz():
    def removeQuestion(questionId):
        FBconn.delete(f'/Questions/{questionId}', None)
        question_element = next((c for c in remove_quiz_container.children if getattr(c, 'question_id', None) == questionId), None)
        if question_element:
            remove_quiz_container.children = tuple(c for c in remove_quiz_container.children if c != question_element)

    def handle_exit_button_click_to_manager(b):
        remove_quiz_container.close()
        exit_button.close()
        overlay.close()
        display(createManagerScreen())

    remove_quiz_container = widgets.VBox(layout=widgets.Layout(width='100%', height='auto', justify_content='center', align_items='center'))
    remove_quiz_container.add_class("background-image")
    
    exit_button = widgets.Button(description="X", layout=widgets.Layout(width='100px',height='auto', align_self='flex-end'))
    exit_button.add_class("button")
    exit_button.on_click(handle_exit_button_click_to_manager)
    
    
    # Create the add question container
    remove_question = widgets.HTML("<img src='https://github.com/EyalKoubi/CloudProject/blob/main/removeQuestion.png?raw=true' align_items='center'>")
    remove_question_container = widgets.HBox([remove_question], layout=widgets.Layout(justify_content='center', align_items='center', margin='20px'))

    question_containers = []

    questions = FBconn.get('/Questions', None)
    if questions:
        for question_id, question_data in questions.items():
            question_container = widgets.HBox(layout=widgets.Layout(width='100%', height='auto', justify_content='center', align_items='center', margin='-10px 0px -10px 0px'))
            question_text = widgets.HTML(f"<p style='font-size:18px; color:#000000; text-align: center;'>{question_data['Question']}</p>")
            remove_button = widgets.Button(description="Remove", layout=widgets.Layout(width='auto',height='auto'))
            remove_button.add_class("button")
            remove_button.question_id = question_id  # Add the question_id attribute
            remove_button.on_click(lambda b, q_id=question_id: removeQuestion(q_id))
            question_container.children = [question_text, remove_button]
            question_container.question_id = question_id
            question_containers.append(question_container)

    remove_quiz_container.children = [exit_button, remove_question_container] + question_containers
    
    # Apply CSS styles
    css = """
    <style>

    .title {
      color: #4169E1;
      text-align: center;
      font-size: 30px;
      padding: 50px 50px;
      text-transform: uppercase;
     }
     
     .button {
        display: inline-block;
        padding: 15px 25px;
        font-size: 12px;
        cursor: pointer;
        text-align: center;
        text-decoration: none;
        outline: none;
        color: #F0F8FF;
        background-color: #6495ED;
        border: none;
        border-radius: 15px;
        box-shadow: 0 9px #4682B4;
        margin: 25px 50px 75px 100px
      }

      .button:hover {background-color: #F0F8FF; color: #6495ED;  margin: 25px 50px 75px 100px}

      .button:active {
        background-color: #4169E1;
        color: #F0F8FF;
        box-shadow: 0 5px #B0E0E6;
        margin: 25px 50px 75px 100px;
        transform: translateY(4px);
      }
        
      .background-image {
        background-image: url('https://w.forfun.com/fetch/3d/3dd0b8f4ab9d0752d3ca9a569723efb3.jpeg?w=1470&r=0.5625');
        background-repeat: no-repeat;
        background-position: center;
        background-size: cover;
      }
      </style>
      """


    overlay.close()
    display(remove_quiz_container)


####################################### manager screen #######################################################################

def createManagerScreen():
    global overlay  # Declare overlay as a global variable

    # Create the overlay widget
    overlay = widgets.Output()

    manager_screen = widgets.VBox()

    with overlay:
        def handle_exit_button_click_to_popup(b):
          overlay.close()
          manager_screen.close()
          display(createWelcomeScreen())

        def handle_add_quiz_button_click(b):
          overlay.close()
          manager_screen.close()
          createAddQuiz()

        def handle_remove_quiz_button_click(b):
          overlay.close()
          manager_screen.close()
          createRemoveQuiz()

        # Create the exit button
        exit_button = widgets.Button(description="X", layout=widgets.Layout(width='100px',height='auto', align_self='flex-end'))
        exit_button.add_class("button")
        exit_container = widgets.HBox([exit_button], layout=widgets.Layout(align_self='flex-end'))
        exit_button.on_click(handle_exit_button_click_to_popup)

        # Create the manager title
        manager_title = widgets.HTML("<img src='https://github.com/EyalKoubi/CloudProject/blob/main/properties.png?raw=true' align_items='center'>")
        title_container = widgets.HBox([manager_title], layout=widgets.Layout(justify_content='center', align_items='center', margin='50px'))
        
        # Create the add quiz button
        add_quiz_button = widgets.Button(description="Add Quiz", layout=widgets.Layout(width='250px', height='auto'))
        add_quiz_button.add_class("button")
        add_quiz_button.on_click(handle_add_quiz_button_click)

        # Create the remove quiz button
        remove_quiz_button = widgets.Button(description="Remove Quiz", layout=widgets.Layout(width='250px', height='auto'))
        remove_quiz_button.add_class("button")
        remove_quiz_button.on_click(handle_remove_quiz_button_click)

        buttons_container = widgets.HBox([add_quiz_button, remove_quiz_button], layout=widgets.Layout(justify_content='center', align_items='center'))

        # Create the manager screen widget
        manager_screen.children = [exit_container, title_container, buttons_container]
        manager_screen.add_class("background-image")

        # Apply CSS styles
        css = """
        <style>

        .title {
          color: #4169E1;
          text-align: center;
          font-size: 30px;
          padding: 50px 50px;
          text-transform: uppercase;
        }

        .button {
          display: inline-block;
          padding: 15px 25px;
          font-size: 24px;
          cursor: pointer;
          text-align: center;
          text-decoration: none;
          outline: none;
          color: #F0F8FF;
          background-color: #6495ED;
          border: none;
          border-radius: 15px;
          box-shadow: 0 9px #4682B4;
          margin: 25px 50px 75px 100px
        }

        .button:hover {background-color: #F0F8FF; color: #6495ED;  margin: 25px 50px 75px 100px}

        .button:active {
          background-color: #4169E1;
          color: #F0F8FF;
          box-shadow: 0 5px #B0E0E6;
          margin: 25px 50px 75px 100px;
          transform: translateY(4px);
        }
        
        .background-image {
            background-image: url('https://w.forfun.com/fetch/3d/3dd0b8f4ab9d0752d3ca9a569723efb3.jpeg?w=1470&r=0.5625');
            background-repeat: no-repeat;
            background-position: center;
            background-size: cover;
        }
        </style>
        """

    # Display the manager screen widget
    display(manager_screen)

    return overlay

####################################### player screen #######################################################################

def createPlayerScreen():

    # Create the player screen widget
    player_screen = widgets.VBox()

    def handle_start_play_button_click(b):
        player_screen.close()
        quizScreen()

    def handle_statistics_button_click(b):
        print("statistics")

    # Create the exit button
    exit_button = widgets.Button(description="X", layout=widgets.Layout(width='auto', height='auto'))
    exit_button.add_class("button")

    # Create the "Statistics" button
    statistics_button = widgets.Button(description="Statistics", layout=widgets.Layout(width='auto', height='auto'))
    statistics_button.add_class("button")
    statistics_button.on_click(handle_statistics_button_click)
    
    # Create button container widget - ilanit
    exit_and_stats_btn = widgets.HBox([statistics_button,exit_button])
    exit_and_stats_btn.add_class("exit_button")

    # Create the player title
    game_info = widgets.HTML("<img src='https://github.com/EyalKoubi/CloudProject/blob/main/gameInfo.png?raw=true' align_items='center'>")

    # Create the "Start Play" button
    start_play_button = widgets.Button(description="PLAY", layout=widgets.Layout(width='250px', height='auto'))
    start_play_button.add_class("button")
    start_play_button.on_click(handle_start_play_button_click)

    # Create screen container widget
    player_screen = widgets.VBox([game_info, start_play_button, exit_and_stats_btn], layout=widgets.Layout(width='100%', height='900px', justify_content='center', align_items='center'))
    player_screen.add_class("screen-container")
    player_screen.add_class("background-image")

    def exit_player_handler(b):
        player_screen.close()
        display(createWelcomeScreen())

    exit_button.on_click(exit_player_handler)

    # Apply CSS styles for rounded buttons
    css = """
    <style>
    .exit_button {
        position: absolute;
        top: 8px;
        right:2px;
    }
    
    .button {
      display: inline-block;
      padding: 15px 25px;
      font-size: 24px;
      cursor: pointer;
      text-align: center;
      text-decoration: none;
      outline: none;
      color: #F0F8FF;
      background-color: #6495ED;
      border: none;
      border-radius: 15px;
      box-shadow: 0 9px #4682B4;
      margin: 25px 50px 75px 100px
    }

    .button:hover {background-color: #F0F8FF; color: #6495ED;  margin: 25px 50px 75px 100px}

    .button:active {
      background-color: #4169E1;
      color: #F0F8FF;
      box-shadow: 0 5px #B0E0E6;
      margin: 25px 50px 75px 100px;
      transform: translateY(4px);
    }
    
    .background-image {
        background-image: url('https://w.forfun.com/fetch/3d/3dd0b8f4ab9d0752d3ca9a569723efb3.jpeg?w=1470&r=0.5625');
        background-repeat: no-repeat;
        background-position: center;
        background-size: cover;
    }
    </style>
    """
    display(HTML(css))

    return player_screen

####################################### activate screens #######################################################################

screen_container = createWelcomeScreen()
display(screen_container)


Output()

Output()

Output()

Output()

Output()